In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [2]:
import pandas as pd
import numpy as np
X_test = pd.read_csv("./yemoonsaBigdata/datasets/Part2/census_X_test.csv")
X_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/census_X_train.csv")
y_train = pd.read_csv("./yemoonsaBigdata/datasets/Part2/census_y_train.csv")

In [ ]:

X_train.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week
count,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,10.121312,1092.007858,88.372489,40.931238
std,13.134665,2.549995,7406.346497,404.298370,11.979984
min,17.000000,1.000000,0.000000,0.000000,1.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000
75%,47.000000,13.000000,0.000000,0.000000,45.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
# 1. 결측치 제거 / 사용할 데이터 선택
for i in range(len(X_train)) :
  if X_train['capital_gain'][i]>0 :
    X_train['capital_gain_yn'][i]=1
  else :
    X_train['capital_gain_yn'][i]=0

In [3]:
X_train['capital_gain_yn']=np.where(X_train['capital_gain']>0,1,0)
X_train['capital_loss_yn']=np.where(X_train['capital_loss']>0,1,0)

X_test['capital_gain_yn']=np.where(X_test['capital_gain']>0,1,0)
X_test['capital_loss_yn']=np.where(X_test['capital_loss']>0,1,0)

In [4]:
col_num=['age', 'education_num','capital_gain_yn', 'capital_loss_yn','hours_per_week']
col_cat=[ 'workclass', 'marital_status', 'occupation','relationship', 'race', 'sex',  'native_country']
col_y=['target']

In [5]:
from sklearn.preprocessing import LabelEncoder

for col in col_cat :
  lb=LabelEncoder()
  lb.fit(X_train[col])
  X_train[col]=lb.transform(X_train[col])
  X_test[col]=lb.transform(X_test[col])

In [6]:
# 2. 데이터 스플릿
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train,y_train,test_size=0.3)

In [7]:
# 3. 프리프로세싱
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler.fit(X_tr[col_num])
X_tr[col_num]=scaler.transform(X_tr[col_num])
X_val[col_num]=scaler.transform(X_val[col_num])
X_test[col_num]=scaler.transform(X_test[col_num])

In [8]:
# 4. 모델링 및 평가
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()
rf.fit(X_tr,y_tr)
pred=rf.predict(X_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [9]:

from sklearn.metrics import roc_auc_score

pred_proba=rf.predict_proba(X_val)

score=roc_auc_score(y_val,pred)
score_proba=roc_auc_score(y_val,pred_proba[:,1])

print('score_pred:',score)
print('score_proba:',score_proba)

score_pred: 0.7775003568991553
score_proba: 0.8965361790965801


In [10]:
from lightgbm import LGBMClassifier
rf=LGBMClassifier()
rf.fit(X_tr,y_tr)
pred=rf.predict(X_val)
from sklearn.metrics import roc_auc_score

pred_proba=rf.predict_proba(X_val)

score=roc_auc_score(y_val,pred)
score_proba=roc_auc_score(y_val,pred_proba[:,1])

print('score_pred:',score)
print('score_proba:',score_proba)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


score_pred: 0.8059963314695994
score_proba: 0.9274526502872876


In [11]:
from xgboost import XGBClassifier
rf=XGBClassifier()
rf.fit(X_tr,y_tr)
pred=rf.predict(X_val)
from sklearn.metrics import roc_auc_score

pred_proba=rf.predict_proba(X_val)

score=roc_auc_score(y_val,pred)
score_proba=roc_auc_score(y_val,pred_proba[:,1])

print('score_pred:',score)
print('score_proba:',score_proba)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


score_pred: 0.7780893059915838
score_proba: 0.91873691096905


In [ ]:
pd.DataFrame({'index': X_test.index, 'target': pred}).to_csv('./yemoonsaBigdata/res/003000000.csv', index=False)